In [ ]:
import sqlite3
import json


# # Initialize the database
# db = sqlite3.connect("file::memory:?cache=shared")
# CREATE_TABLE_SQL = "CREATE TABLE test_table(json_data TEXT)"
# with db:
#     cur = db.cursor()
#     cur.execute(CREATE_TABLE_SQL)

# # Insert some data
# obj_a = dict(name="test a")
# obj_b = dict(name="test b")

# INSERT_SQL = "INSERT INTO test_table VALUES (?)"
# with db:
#     cur = db.cursor()
#     cur.execute(INSERT_SQL, (json.dumps(obj_a),))
#     cur.execute(INSERT_SQL, (json.dumps(obj_b),))


obj_a = dict(name="test a", a_specific="a", common=dict(a=1, b=2))
obj_b = dict(name="test b", b_specific="b", common=dict(b=20, c=30))
db = sqlite3.connect("file::memory:?cache=shared")
with db:
    # cur.execute("SELECT json_patch(?, ?)", ('{"name":"obj-1"}', '{"names": "obj-2"}'))
    # print(cur.fetchall())
    cur.execute(f"SELECT json_patch(?, ?)", (json.dumps(obj_a), json.dumps(obj_b)))
    print(cur.fetchall())